In [12]:
from selenium import webdriver
import requests
import io
from PIL import Image
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import os
import shutil

In [2]:
path = "C:\\Users\\aryam\\Documents\\ML\\ImageToRecipe\\chromedriver.exe"
cService = webdriver.ChromeService(executable_path=path)
# driver = webdriver.Chrome(service = cService)
# driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")

In [7]:
def download_image(download_path,url,filename):
    try:
        image_content = requests.get(url).content
        image_bytes = io.BytesIO(image_content)
        file_path = download_path + filename
        image = Image.open(image_bytes)
        with open(file_path,"wb") as f:
            image.save(f,"JPEG")
    except Exception as e:
        print("Exception is ", e)

In [8]:
def download_images(download_folder_path,urls,recipe,recipe_index,train_imgs_count):
    try:
        recipe_folder_name = str(recipe_index) + "_" + recipe
        train_folder_path = os.path.join(download_folder_path,recipe_folder,"train")
        test_folder_path = os.path.join(download_folder_path,recipe_folder,"test")

        if not os.path.exists(train_folder_path):
            os.makedirs(train_folder_path)
        if not os.path.exists(test_folder_path):
            os.makedirs(test_folder_path)
            
        img_number = 1
        for url in urls:
            if img_number <= train_imgs_count:
                download_image(train_folder_path,url,str(img_number)+"_"+recipe+".jpg")
            else:
                download_image(test_folder_path,url,str(img_number)+"_"+recipe+".jpg")
            img_number+=1
    except Exception as e:
        print("Exception is ", e)

In [ ]:
# download_image("",image_url,"radha_krsna.jpg")
# sFlh5c pT0Scc iPVvYb
# thumbnail - mNsIhb

In [6]:
p  = os.path.join(".\\downloaded_images\\train","folder2")
if not os.path.exists(p):
    os.makedirs(p)

In [46]:
driver = webdriver.Chrome(service=cService)
driver.maximize_window()
driver.get('https://images.google.com/')
search_box = driver.find_element(By.NAME,"q")
time.sleep(7)
search_box.send_keys("car")
search_box.send_keys(Keys.ENTER)
start = 10
length = 10
time.sleep(7)
thumbnails = driver.find_elements(By.CLASS_NAME,"mNsIhb")
print(len(thumbnails))
try:
    print(thumbnails[0].get_attribute("id"))
    thumbnails[0].click()
    time.sleep(7)
    # pop_box = driver.find_element(By.ID,"Sva75c")
    # if pop_box != None:
    #     print("found")
    # pop_box.send_keys(Keys.CONTROL + 'w')
    elements = driver.find_elements(By.CLASS_NAME,"jlTjKd")
    print(len(elements))
    for elem in elements:
        if elem.tag_name == 'a':
            print("found  a ")
            imgs= elem.find_elements(By.TAG_NAME,"img")
            print(len(imgs))
            for img in imgs:
                class_name = img.get_attribute("class")
                if len(class_name.split()) >=3 :
                    third_part = class_name.split()[2]
                    if third_part == "iPVvYb":
                        print(img.get_attribute("src"))          
    # img_container = elements.find_elements(By.TAG_NAME,"a")
    # print(len(img_container))
    # for img_box in img_container :
    #     img = img_box.find_element(By.TAG_NAME,"img")
    #     print(img.get_attribute("src"))
    #     print(img.get_attribute("jsname"))
    #     print()
except Exception as e:
    print(e)

83

2
found  a 
2
https://imgd.aeplcdn.com/272x153/n/cw/ec/40432/scorpio-n-exterior-right-front-three-quarter-75.jpeg?isig=0&q=80
found  a 
1


In [ ]:
df = pd.read_csv("links.csv")

In [10]:
def create_folder(folder_path):
    pass

In [ ]:
try :
    # Prepare driver
    driver = webdriver.Chrome(service=cService)
    driver.maximize_window()
    driver.get('https://images.google.com/')
    time.sleep(3)
    search_box = driver.find_element(By.NAME,"q")

    # initialize counters and delays
    start = 0
    count= 10
    big_delay = 7
    small_delay = 3
    max_imgs = 10
    extra_imgs = 5

    # load links.csv
    df = pd.read_csv("links_copy.csv",skiprows= start,nrows=count,names=['name','link'])

    # open logs
    f = open("download_logs.txt","a")

    # Iterate recipies
    for i in range(len(df['name'])):
        recipie = df.loc[i,'name']

        # make entry to log 
        print(start+i,recipie)
        f.write(str(start+i)+" "+ recipie +"\n")
        f.flush()

        # Enter recipie name in box and hit ENTER
        search_box.send_keys(recipie)
        search_box.send_Keys(Keys.ENTER)
        time.sleep(big_delay)

        # Fill thumbnails
        thumbnails = driver.find_elements(By.CLASS_NAME,"mNsIhb")

        while len(thumbnails) < max_imgs+extra_imgs: # checking condition before scrolling to minimize HTTP Requests
            print("Less images obtained")
            time.sleep(small_delay)
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            time.sleep(small_delay)
            thumbnails = driver.find_elements(By.CLASS_NAME,"mNsIhb") 

        # Click on Each thumbnail and Extract src
        image_urls = set()
        success_count = 0
        for thumbnail in thumbnails:
            if success_count == max_imgs:
                break

            try:
                thumbnail.click()
                time.sleep(small_delay)
            except:
                print("Error in clicking thumbnail")
                continue
            
            # Find the pop-up image box
            pop_up_box_elems = driver.find_elements(By.CLASS_NAME,"jlTjKd")
            
            for elem in pop_up_box_elems:

                # find <a> tag , it has the img
                if elem.tag_name == 'a':

                    # get all the img tags under <a>
                    img_tags = elem.find_elements(By.TAG_NAME,"img")

                    for img in img_tags:

                        # get the attribute of that img tag
                        # two very similar but different <img> tags would be present
                        class_name = img.get_attribute("class")

                        # split and the check the third class name
                        if len(class_name.split()) >=3 :
                            third_part = class_name.split()[2]
                            if third_part == "iPVvYb":
                                img_url = img.get_attribute("src")

                                if img_url not in image_urls :
                                    image_urls.add(img_url)
                                    success_count += 1
                                    f.write("Found " + success_count + "\n")
                                else:
                                    f.write("Found Duplicate" + "\n")
                                f.flush()
            time.sleep(big_delay)
        
        f.write("\n")
        f.flush()
        time.sleep(big_delay)                   





        
except Exception as e:
    print("Exception occurred ", e)


In [13]:
def handler(func,path,exec_info):
    pass

try:
    shutil.rmtree(".\\downloaded_images\\train\\folder1",onerror=handler)
except Exception as e:
    print('exception',e)

In [14]:
x = input("are you happy")